# Facebook meesenger tour bot implementation

 - Remeber to run a tunneler like ngrok
 - Fill in the URL in the Facebook webhook

### First set up the LLM - this is the 7 billion version for now

In [1]:
from torch import cuda, bfloat16
import transformers

#Select a model to load in
model_id = 'meta-llama/Llama-2-7b-chat-hf'

#Check if the GPU is detected
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

# Begin initializing HF items, need auth token for these (fill in your personal code-this is mine)
hf_auth = '##HF token removed'
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

# IDK probably like load the model XD
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)
model.eval()
print(f"Model loaded on {device}")

#Load in the model tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

#Create a function used to generate text response
generate_text = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    temperature=0.1,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=250,  # mex number of tokens to generate in the output
    repetition_penalty=1.1,  # without this output begins repeating
    batch_size=15      #Try to change it to avoid the out of memory error
)

/home/hubby/anaconda3/envs/TextGen/lib/python3.11/site-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model loaded on cuda:0


/home/hubby/anaconda3/envs/TextGen/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [ ]:
#Create a function used to generate text response
generate_text = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    temperature=0.1,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=250,  # mex number of tokens to generate in the output
    repetition_penalty=1.1,  # without this output begins repeating
    batch_size=15      #Try to change it to avoid the out of memory error
)

In [2]:
#Set the basic prompt to make it give a reasonable answer
prompt="""Assistant is an expert of art and provides answers to any questions about it a person might have. They do not want to bore the intrerlocutor, so their response is never longer than 300 words.

Here are some previous conversations between the Assistant and User:

User: Hey how are you today?
Assistant: ```Hi! I'm great as always, ready to hear your questions.
```
User: How can the Stańczyk painting be interpreted?
Assistant: ```
The painting of Stańczyk can be interpreted in various ways, but above all, it portrays Matejko as an artist engaged in the political life of his time. Stańczyk predicts the consequences of the mistakes of Polish rulers, which will lead to the rise of Moscow's power at the expense of weakening Poland and Lithuania.
```
User: What do Adam and Eve represent in this painting?
Assistant: ```In this painting, which is titled "The Garden of Eden," Adam and Eve represent a significant biblical and symbolic narrative. They are central characters in the story of creation and the origin of humanity, as described in the book of Genesis in the Bible.

Adam, typically depicted as the first man, and Eve, the first woman, are often seen as symbols of the first human beings created by God. They are placed in the idyllic setting of the Garden of Eden, which represents a paradise and a state of innocence.

Their actions in the story carry deep symbolic meaning. Adam and Eve are often seen as symbols of the fall of humanity due to their disobedience to God's command not to eat from the Tree of Knowledge of Good and Evil. Their decision to eat the forbidden fruit led to their expulsion from Eden and introduced sin and mortality into the world.
```

Here is the question of the visitor: {question} ; Your response:"""

In [8]:
incoming_message='Hi, how are you!'
res = generate_text(prompt.format(question=incoming_message))
print('Response generated')
#Return only the response, not the prompt
        #start with getting the len of the prompt
prompt_len = len(prompt)
        #Take the right part
answer = res[0]["generated_text"].split('response: ')[1]
print(answer)

Response generated
```Hi! I'm great as always, ready to hear your questions.


### Run the Flask app to connect to Messenger

In [9]:
from flask import Flask, request
import requests

from pymessenger import Bot
 
app = Flask(__name__)


PAGE_ACCESS_TOKEN = '' #Removed 4 github
# This is API key for facebook messenger.
API = "https://graph.facebook.com/LATEST-API-VERSION/me/messages?access_token="+PAGE_ACCESS_TOKEN

#This is an unofficial package which makes it very easy to send stuff through messenger
bot = Bot(PAGE_ACCESS_TOKEN)

#Function to respond to a verification request from Messenger
@app.route("/", methods=['GET'])
def fbverify():
    
    if not request.args.get("hub.verify_token")== "verifytokenstring":
        print("SOMETHING BAD")
        return "Verification token mismatch", 403
    print('SEEMS GOOD')
    return request.args['hub.challenge'], 200
    

#Function - what to do on a received POST, which in this case is a message to the bot.
@app.route("/", methods=['POST'])
def fbwebhook():
    #Get the request
    data = request.get_json()
    print(data)
    try:
        # Read messages from facebook messanger.
        #print('No error')
        message = data['entry'][0]['messaging'][0]['message']
        sender_id = data['entry'][0]['messaging'][0]['sender']['id']

        #Respond to the message:

        #For testing purposes
        if message['text'] == "hi":
            bot.send_text_message(sender_id,'DID IT GO THROUGH (but hi)?')
            return 'test1' #response

        #Use LLM to generate the answer
        else:
            resp = generate_response(message)
            bot.send_text_message(sender_id,resp)
            return "TEST" #response

    except KeyError:        
        print("PRZESZŁO W ERROR")
        pass
    
def generate_response(incoming_message):
    #Actually generate the response
    print('Starting response generation')
    res = generate_text(prompt.format(question=incoming_message))
    print('Response generated')
    #Return only the response, not the prompt
        #start with getting the len of the prompt
    prompt_len = len(prompt)
        #Take the right part
    answer = res[0]["generated_text"].split('response: ')[1]
    print(answer)
    return answer

    
if __name__ =='__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


{'object': 'page', 'entry': [{'id': '146598931863670', 'time': 1696243644487, 'messaging': [{'sender': {'id': '7286765631368676'}, 'recipient': {'id': '146598931863670'}, 'timestamp': 1696243216664, 'message': {'mid': 'm_3wRztuY_nCQGQx0UVWlJSNTh-sUKOD1wN0KS908IuZIPvBKTHLXobGI3gODZwqou9PMREjE1f67biP-H0W3d_w', 'text': 'Hi sir, who are you?'}}]}]}
Starting response generation
Response generated
```Hi there! I'm just an AI Assistant trained to provide information and answer questions about art. I don't have personal experiences or feelings like humans do, but I'm here to help you learn more about art and its many wonders. Is there something specific you would like to know or discuss?


127.0.0.1 - - [02/Oct/2023 12:47:29] "POST / HTTP/1.1" 200 -


{'object': 'page', 'entry': [{'id': '146598931863670', 'time': 1696243649173, 'messaging': [{'sender': {'id': '7286765631368676'}, 'recipient': {'id': '146598931863670'}, 'timestamp': 1696243622686, 'message': {'mid': 'm_7rEvA8tZ_9ym6Og_77OHsdTh-sUKOD1wN0KS908IuZJMSu3g_geQ-dxv3RV_AeZ4FhZvOQJQzjDoyuPL4LLH_g', 'text': 'Welcome back!'}}]}]}
Starting response generation
Response generated
```Thank you for coming back! I'm here to help with any questions or topics you'd like to discuss related to art. Please feel free to ask me anything.
```


...

127.0.0.1 - - [02/Oct/2023 13:54:05] "POST / HTTP/1.1" 200 -


{'object': 'page', 'entry': [{'id': '146598931863670', 'time': 1696247736369, 'messaging': [{'sender': {'id': '7286765631368676'}, 'recipient': {'id': '146598931863670'}, 'timestamp': 1696247735905, 'message': {'mid': 'm_MqVWJh24PmJDlWhV-wf_u9Th-sUKOD1wN0KS908IuZK_hN7mdqRNyzqXr_4BFtJPhPo1V_wWtZ2D0yOdF1yzCw', 'text': 'Who is Van Gogh?'}}]}]}
Starting response generation
Response generated
```Vincent van Gogh (1853-1890) was a Dutch post-impressionist painter known for his bold and expressive use of color and brushwork. He is widely considered one of the greatest painters in history, and his work has had a profound influence on modern art.
Born in the Netherlands, Van Gogh began his career as an art dealer before turning to painting in his mid-twenties. Despite struggling with mental illness and poverty throughout his life, he produced over 2,000 artworks, including famous paintings such as "Sunflowers," "The Starry Night," and "Self-Portrait with Bandaged Ear."
Van Gogh's innovative sty

127.0.0.1 - - [02/Oct/2023 13:55:49] "POST / HTTP/1.1" 200 -


{'object': 'page', 'entry': [{'id': '146598931863670', 'time': 1696249222221, 'messaging': [{'sender': {'id': '7286765631368676'}, 'recipient': {'id': '146598931863670'}, 'timestamp': 1696249221692, 'message': {'mid': 'm_NufO6RL9YSDt8O3C5jjpzNTh-sUKOD1wN0KS908IuZKwH5NQSromuiBmxoQKRcZ7kXBb21OzUA8A_ssdgF-mtg', 'text': "What was Rembrandt's childhood like? Tell me in 50 words"}}]}]}
Starting response generation
Response generated
```Rembrandt van Rijn was born in 1606 in Leiden, Netherlands. His father, a miller, died when he was just seven years old, leaving his mother to raise him and his four siblings. Despite these challenges, Rembrandt's mother encouraged his artistic pursuits, and he began his training as an artist at a young age.
```


127.0.0.1 - - [02/Oct/2023 14:20:28] "POST / HTTP/1.1" 200 -
